In [2]:
import numpy as np 
import pandas as pd

## INPUT DATA

In [170]:
types_of_loadings = ['y_conc','moment','C_dist','L_dist','x_conc','D_axial']
## y_conc = carga concentrada en y ingresar el valor de la carga (+ en direccion contraria al eje y) y el valor de la posicion
## con respecto al primer nodo

##moment = momento aplicado ingresar el valor del momento aplicado (+ en el sentido de las agujas del reloj) y la posicion
## con respecto al primer nodo.

##C_dist = carga distribuida uniformemente (constante) ingresar el valor de la carga distribuida y la posicion con respecto
## al primer y segundo nodo en forma de tupla ()

## L_dist = carga distribuida en forma lineal. ingresar el valor de el valor minimo y maximo de la carga distribuida en forma de
## tupla () y la posicion de esas magnitudes en forma de tupla () empezar desde el primer nodo, las posiciones son l1: desde el
## primer nodo, l2: desde el segundo nodo.

## x_conc = carga caoncertrada axial, ingresar la magnitud de la carga puntual axial y la posicion de esta carga desde el primer
## nodo

## D_axial = carga distribuida axial, ingresar el valor de la carga dist axial y las posiciones de igual manera que las cargas
## distribuidas de manera perpendicular al eje del elemento

joint_data = pd.DataFrame(np.array([[1,0,0],[2,120,240],[3,360,240]]),columns=['joint','x','y'])
suport_data = pd.DataFrame(np.array([[1,1,1,1],[3,1,1,1]]),columns = ['suport','x','y','r'])
materialp_data = pd.DataFrame(np.array([[1,29000]]),columns=['type','E'])
crossec_data = pd.DataFrame(np.array([[1,11.8,310]]),columns=['type','area','inertia'])
member_data = pd.DataFrame(np.array([[1,1,2,1,1],[2,2,3,1,1],]),columns=['member','start','end','material_type','cross_type'])
jointload_data = pd.DataFrame(np.array([[2,0,0,-1500]]),columns = ['joint','x_load','y_load','m_load'])
memberload_data = pd.DataFrame(np.array([[1,'x_conc',80.498,268.33/2],[1,'y_conc',40.249,268.33/2],
                                        [2,'C_dist',0.125,(0,0)]]),columns = ['member','loadtype','w','position'])


memberload_data.member = memberload_data.member.astype('int')

names = zip(['joint_data','suport_data','materialp_data','crossec_data','member_data','jointload_data','memberload_data'],
             [joint_data,suport_data,materialp_data,crossec_data,member_data,jointload_data,memberload_data])

for i,a in names:
    print(i,'\n',a)
    print('\n')

joint_data 
    joint    x    y
0      1    0    0
1      2  120  240
2      3  360  240


suport_data 
    suport  x  y  r
0       1  1  1  1
1       3  1  1  1


materialp_data 
    type      E
0     1  29000


crossec_data 
    type  area  inertia
0   1.0  11.8    310.0


member_data 
    member  start  end  material_type  cross_type
0       1      1    2              1           1
1       2      2    3              1           1


jointload_data 
    joint  x_load  y_load  m_load
0      2       0       0   -1500


memberload_data 
    member loadtype       w position
0       1   x_conc  80.498  134.165
1       1   y_conc  40.249  134.165
2       2   C_dist   0.125   (0, 0)




<ipython-input-170-a544c6ccf9fb>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  memberload_data = pd.DataFrame(np.array([[1,'x_conc',80.498,268.33/2],[1,'y_conc',40.249,268.33/2],


## SETTING PARAMETERS

In [171]:
ncjt = 3
nj = len(joint_data)
ns = len(suport_data)
rc = (suport_data[['x','y','r']] == 1).sum().sum()
ndof = (ncjt*nj)-rc
rc

6

In [172]:
def structure_numbers():
    k = ndof
    j=0
    str_number = np.zeros(ncjt*nj).reshape(ncjt*nj,1)
    for i in range(1,nj+1):
        if any(suport_data.suport==i):
            row = suport_data[suport_data.suport == i]
            
            if row.x.values == 1:
                k+=1
                str_number[(ncjt*i)-3]=k
            else:
                j+=1
                str_number[(ncjt*i)-3]=j
                
            if row.y.values == 1:
                k+=1
                str_number[(ncjt*i)-2]=k
            else:
                j+=1
                str_number[(ncjt*i)-2]=j
                
            if row.r.values == 1:
                k+=1
                str_number[(ncjt*i)-1]=k
            else:
                j+=1
                str_number[(ncjt*i)-1]=j
        else:
            j+=1
            str_number[(ncjt*i)-3]=j
            j+=1
            str_number[(ncjt*i)-2]=j
            j+=1
            str_number[(ncjt*i)-1]=j
            
    return str_number.astype('int64')

str_number = structure_numbers()
str_number

array([[4],
       [5],
       [6],
       [1],
       [2],
       [3],
       [7],
       [8],
       [9]], dtype=int64)

## MEMBER PARAMETERS, TRANSFORMATION MATRIX, LOCAL AND GLOBAL STIFFNESS MATRICES

In [173]:
def member_parameters(member_number):
    info = member_data[member_data.member==member_number]
    start = info.start.item()
    end = info.end.item()
    E = materialp_data[materialp_data.type==info.material_type.item()].E.item()
    A = crossec_data[crossec_data.type==info.cross_type.item()].area.item()
    I = crossec_data[crossec_data.type==info.cross_type.item()].inertia.item()
    x_b = joint_data[joint_data.joint==start].x.item() 
    x_e = joint_data[joint_data.joint==end].x.item() 
    y_b = joint_data[joint_data.joint==start].y.item() 
    y_e = joint_data[joint_data.joint==end].y.item() 
    l = np.sqrt((x_e-x_b)**2+(y_e-y_b)**2)
    costheta = (x_e-x_b)/l
    sintheta = (y_e-y_b)/l
    membercode_number=np.array([str_number[ncjt*start-3].item(),str_number[ncjt*start-2].item(),str_number[ncjt*start-1].item(),
                                str_number[ncjt*end-3].item(),str_number[ncjt*end-2].item(),str_number[ncjt*end-1].item()]).reshape(2*ncjt,1)
    
    return (E,A,I,l,costheta,sintheta,membercode_number)

def transformation_matrix(member_number):
    _,_,_,_,costheta,sintheta,_ = member_parameters(member_number)
    T = np.array([costheta,sintheta,0,0,0,0,
                  -sintheta,costheta,0,0,0,0,
                  0,0,1,0,0,0,
                  0,0,0,costheta,sintheta,0,
                  0,0,0,-sintheta,costheta,0,
                  0,0,0,0,0,1]).reshape(6,6)
    return T

def local_and_global_stiffness_matrix(member_number):
    E,A,I,l,costheta,sintheta,membercode_number = member_parameters(member_number)
    stiffness = E*I/l**3
    T = transformation_matrix(member_number)
    a1 =(A*l**2)/I
    a2 = 4*l**2
    a3=2*l**2
    k_local = stiffness*np.array([a1,0,0,-a1,0,0,
                                  0,12,6*l,0,-12,6*l,
                                  0,6*l,a2,0,-6*l,a3,
                                  -a1,0,0,a1,0,0,
                                  0,-12,-6*l,0,12,-6*l,
                                  0,6*l,a3,0,-6*l,a2]).reshape(6,6)
    k_global = np.dot(np.dot(T.T,k_local),T)
    return (k_local,k_global,membercode_number)

member_parameters(1)[-1]
a= local_and_global_stiffness_matrix(1)[1]
b= local_and_global_stiffness_matrix(2)[1]
b

# for i in members:
#     print('--------------member no.{}--------------------'.format(i),'\n')
#     print('member_parameters','\n',member_parameters(i),'\n','\n')
#     print('transformation','\n',transformation_matrix(i),'\n','\n')
#     print('local_and_global','\n',local_and_global_stiffness_matrix(i),'\n','\n','\n')

array([[ 1.42583333e+03,  0.00000000e+00,  0.00000000e+00,
        -1.42583333e+03,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  7.80381944e+00,  9.36458333e+02,
         0.00000000e+00, -7.80381944e+00,  9.36458333e+02],
       [ 0.00000000e+00,  9.36458333e+02,  1.49833333e+05,
         0.00000000e+00, -9.36458333e+02,  7.49166667e+04],
       [-1.42583333e+03,  0.00000000e+00,  0.00000000e+00,
         1.42583333e+03,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -7.80381944e+00, -9.36458333e+02,
         0.00000000e+00,  7.80381944e+00, -9.36458333e+02],
       [ 0.00000000e+00,  9.36458333e+02,  7.49166667e+04,
         0.00000000e+00, -9.36458333e+02,  1.49833333e+05]])

## STRUCTURE STIFFNESS MATRIX

In [174]:
def structure_stiffness_matrix():
    members = len(member_data)
    S = np.zeros(ndof*ndof).reshape(ndof,ndof)
    for member in range (1,members+1):
        k_local,k_global,membercode_number = local_and_global_stiffness_matrix(member)

        for i in range(1,(2*ncjt)+1):
            n1 = membercode_number[i-1]
            if n1 <= ndof:
                for j in range(1,(2*ncjt)+1):
                    n2 = membercode_number[j-1]
                    if n2 <= ndof:
                        S[n1-1,n2-1] = S[n1-1,n2-1] + k_global[i-1,j-1]
                    else:
                        continue
            else:
                continue
    return S

structure_stiffness_matrix()

array([[  1685.36132088,    507.88805781,    670.07503726],
       [   507.88805781,   1029.16389371,    601.4208147 ],
       [   670.07503726,    601.4208147 , 283848.34078482]])

## JOINT LOAD VECTOR

In [175]:
def joint_load_vector():
    p = np.zeros(ndof).reshape(ndof,1)
    rows = len(jointload_data)
    for i in range(rows):
        loaded_joint = int(jointload_data.iloc[i].joint.item())
        Xstr_number = str_number[(ncjt*loaded_joint)-3].item()
        Ystr_number = str_number[(ncjt*loaded_joint)-2].item()
        Rstr_number = str_number[(ncjt*loaded_joint)-1].item()
        if Xstr_number <= ndof:
            p[Xstr_number-1] = jointload_data.iloc[i].x_load.item()
        else:
            None

        if Ystr_number <= ndof:
            p[Ystr_number-1] = jointload_data.iloc[i].y_load.item()
        else:
            None
        if Rstr_number <= ndof:
            p[Rstr_number-1] = jointload_data.iloc[i].m_load.item()
    return p

joint_load_vector()

array([[    0.],
       [    0.],
       [-1500.]])

## MEMBER AND STRUCTURE EQUIVALENT JOINT LOAD VECTOR

In [176]:
def member_equivalent_joint_loadvector(member):
    info = memberload_data[memberload_data.member == member]
    Qf= np.zeros(2*ncjt).reshape(2*ncjt,1)
    for i in range(len(info)):
        lp = info.iloc[i].position
        load_type = info.iloc[i].loadtype
        w = info.iloc[i].w
        _,_,_,l,_,_,membercode_number = member_parameters(member)

        if load_type == types_of_loadings[0]:
            v1 = ((w/l**3)*(l-lp)**2)*((3*lp)+(l-lp))
            m1 = (w/l**2)*(lp*(l-lp)**2)
            v2 = ((w/l**3)*lp**2)*(lp+(3*(l-lp)))
            m2 = -1*(w/l**2)*((lp**2)*(l-lp))
            Qf = Qf + np.array([[0],[v1],[m1],[0],[v2],[m2]])

        elif load_type == types_of_loadings[1]:
            v1 = (-1*6*w/l**3)*(lp*(l-lp))
            m1 = ((w/l**2)*(l-lp))*((l-lp)-(2*lp))
            v2 = (6*w/l**3)*(lp*(l-lp))
            m2 = ((w/l**2)*lp)*(lp-(2*(l-lp)))
            Qf = Qf + np.array([[0],[v1],[m1],[0],[v2],[m2]])

        elif load_type == types_of_loadings[2]:
            v1 = (w*l/2)*(1-((lp[0]/l**4)*((2*l**3)-((2*lp[0]**2)*l)+lp[0]**3))-(((lp[1]**3)/l**4)*((2*l)-lp[1])))
            m1 = ((w*l**2)/12)*(1-(((lp[0]**2)/(l**4))*(6*l**2-8*lp[0]*l+3*lp[0]**2))-(((lp[1]**3)/(l**4))*(4*l-3*lp[1])))                                                                                       
            v2 = (w*l/2)*(1-(((lp[0]**3)/(l**4))*(2*l-lp[0]))-((lp[1]/l**4)*(2*l**3-((2*lp[1]**2)*l)+lp[1]**3)))
            m2 = (-1*(w*l**2)/12)*(1-(((lp[1]**2)/(l**4))*(6*l**2-8*lp[1]*l+3*lp[1]**2))-(((lp[0]**3)/(l**4))*(4*l-3*lp[0])))
            Qf = Qf + np.array([[0],[v1],[m1],[0],[v2],[m2]])

        elif load_type == types_of_loadings[3]:
            a = (7*l+8*lp[0])-(((lp[1]*(3*l+2*lp[0]))/(l-lp[0]))*(1+((lp[1])/(l-lp[0]))+((lp[1]**2)/(l-lp[0])**2)))+((2*lp[1]**4)/(l-lp[0])**3) 
            b = ((3*l+2*lp[0])*(1+((lp[1])/(l-lp[0]))+((lp[1]**2)/(l-lp[0])**2)))-(((lp[1]**3)/(l-lp[0])**2)*(2+((15*l-8*lp[1])/(l-lp[0]))))
            c = (3*l+12*lp[0])-(((lp[1]*(2*l+3*lp[0]))/(l-lp[0]))*(1+((lp[1])/(l-lp[0]))+((lp[1]**2)/(l-lp[0])**2)))+((3*lp[1]**4)/(l-lp[0])**3)
            d = ((2*l+3*lp[0])*(1+((lp[1])/(l-lp[0]))+((lp[1]**2)/(l-lp[0])**2)))-(((3*lp[1]**3)/(l-lp[0])**2)*(1+((5*l-4*lp[1])/(l-lp[0]))))
            v1 = a*((w[0]*(l-lp[0])**3)/(20*l**3))+b*((w[1]*(l-lp[0])**3)/(20*l**3))
            m1 = c*((w[0]*(l-lp[0])**3)/(60*l**2))+d*((w[1]*(l-lp[0])**3)/(60*l**2))
            v2 = (((w[0]+w[1])/2)*(l-lp[0]-lp[1]))-v1
            m2 = (((l-lp[0]-lp[1])/6)*((w[0]*(-2*l+2*lp[0]-lp[1]))-(w[1]*(l-lp[0]+2*lp[1]))))+(v1*l)-m1
            Qf = Qf + np.array([[0],[v1],[m1],[0],[v2],[m2]])
        
        elif load_type == types_of_loadings[4]:
            A1 = (w*(l-lp))/l 
            A2 = (w*lp)/l
            Qf = Qf + np.array([[A1],[0],[0],[A2],[0],[0]])
            
        elif load_type == types_of_loadings[5]:
            A1 = (w/(2*l))*(l-lp[0]-lp[1])*(l-lp[0]+lp[1])
            A2 = (w/(2*l))*(l-lp[0]-lp[1])*(l+lp[0]-lp[1])
            Qf = Qf + np.array([[A1],[0],[0],[A2],[0],[0]])
            
        else:
            print('bad inputs in: {}'.format(load_type))
            
        
    return Qf


def structure_equivalent_joint_loadvector():
    pf = np.zeros(ndof).reshape(ndof,1)
    for m in set(memberload_data.member.values):
        _,_,_,_,_,_,membercode_number = member_parameters(m)
        T = transformation_matrix(m)
        Qf = member_equivalent_joint_loadvector(m)
        Ff= np.dot(T.T,Qf)
        for i,a in zip(membercode_number.reshape(-1,),list(range(len(Qf)))):
            if i <= ndof:
                pf[i-1] = pf[i-1] + Ff[a].item()
            else:
                None
    return pf

structure_equivalent_joint_loadvector()

array([[-6.18056944e-05],
       [ 6.00000899e+01],
       [-7.50001771e+02]])

## JOINT DISPLACEMENTS

In [177]:
displacements = np.dot(np.linalg.inv(structure_stiffness_matrix()),(joint_load_vector() - structure_equivalent_joint_loadvector()))
displacements

array([[ 0.02130148],
       [-0.06732193],
       [-0.00254989]])

## REACTIONS

In [186]:
members = len(member_data)
reactions = np.zeros(rc).reshape(rc,1)
for member in range(1,members+1):
    _,_,_,_,_,_,membercode_number = member_parameters(member)
    T = transformation_matrix(member)
    k_local,_,_ =local_and_global_stiffness_matrix(member) 
    membglobal_disp = np.zeros(2*ncjt).reshape(2*ncjt,1)
    for i in range(2*ncjt):
        n = membercode_number[i].item()
        if n <= ndof:
            membglobal_disp[i]=displacements[membercode_number[i].item()-1]
        else:
            None
            
    memblocal_disp = T.dot(membglobal_disp)
    memblocal_forces = k_local.dot(memblocal_disp)+member_equivalent_joint_loadvector(member)
    membglobal_forces = T.T.dot(memblocal_forces)
    for i in range(2*ncjt):
        n = membercode_number[i].item()
        if n > ndof:
            reactions[n-ndof-1]=reactions[n-ndof-1]+membglobal_forces[i]
        else:
            None
reactions

array([[  30.37235513],
       [ 102.08626295],
       [1215.95019753],
       [ -30.37235513],
       [  17.91323708],
       [-854.07369273]])

## DISPLAY

In [187]:
def joint_displacements():
    jt_displacements = np.zeros(ncjt*nj).reshape(ncjt*nj,1)

    for i in range(ncjt*nj):
        n= str_number[i]
        if n <= ndof:
            jt_displacements[i] = displacements[n-1]
        else:
            jt_displacements[i] = 0
    jt_displacements.reshape(nj,-1)
    joint_no = joint_data.joint.values
    final = pd.DataFrame(jt_displacements.reshape(nj,-1),columns=['x_disp','y_disp','rot'],index =joint_no )
    final.index.name = 'joint.no'
    return (jt_displacements,final)


def suport_reactions():
    sup_reactions = np.zeros(ncjt*ns).reshape(ncjt*ns,1)
    j = 0        
    for i in suport_data.suport:
        j+=ncjt
        n1 = str_number[ncjt*i-3].item()
        n2 = str_number[ncjt*i-2].item()
        n3 = str_number[ncjt*i-1].item()
        if n1> ndof:
            sup_reactions[j-3] = reactions[n1-ndof-1]
        else:
            None
        if n2 > ndof:
            sup_reactions[j-2] = reactions[n2-ndof-1]
        else:
            None
        if n3 > ndof:
            sup_reactions[j-1] = reactions[n3-ndof-1]
        else:
            None
    suport_joints = suport_data.suport.values
    final = pd.DataFrame(sup_reactions.reshape(ns,-1),columns=['x_reac','y_reac','moment'], index=suport_joints)
    final.index.name = 'joint.no'
    return final


def internal_forces():
    index = pd.MultiIndex(levels=[[],[]],codes=[[],[]],names=[u'member', u'joint'])
    int_forces = pd.DataFrame(index= index,columns=['axial_force','shear','moment'])
    jd,_ = joint_displacements()
    for member in member_data.member.values:
        start = member_data[member_data.member == member].start.item()
        end = member_data[member_data.member == member].end.item()
        T = transformation_matrix(member)
        k_local,k_global,_ = local_and_global_stiffness_matrix(member)
        Qf = member_equivalent_joint_loadvector(member)
        global_disp = np.array([jd[(ncjt*start)-3],jd[(ncjt*start)-2],jd[(ncjt*start)-1],
                                jd[(ncjt*end)-3],jd[(ncjt*end)-2],jd[(ncjt*end)-1]])
        local_disp =  np.dot(T,global_disp)
        local_forces = np.dot(k_local,local_disp)+ Qf
        local_forces = local_forces.reshape(-1,ncjt)
        int_forces.loc[(member,start),:] = local_forces[0]
        int_forces.loc[(member,end),:] = local_forces[1] 
    
    return int_forces


print(joint_displacements()[1],'\n','\n',suport_reactions(),'\n','\n',internal_forces())

            x_disp    y_disp      rot
joint.no                             
1         0.000000  0.000000  0.00000
2         0.021301 -0.067322 -0.00255
3         0.000000  0.000000  0.00000 
 
              x_reac      y_reac       moment
joint.no                                    
1         30.372355  102.086263  1215.950198
3        -30.372355   17.913237  -854.073693 
 
              axial_force    shear   moment
member joint                              
1      1         104.892  18.4885  1215.95
       2        -24.3937  21.7605  -1654.9
2      2         30.3724  12.0868  154.897
       3        -30.3724  17.9132 -854.074


In [2]:
K_str = np.zeros([ncjt*nj,ncjt*nj])
for n in joint_data.joint.values:
    members = member_data[(member_data.start == n)|(member_data.end == n)]

    for mem in range(len(members)):

        start = members.iloc[mem].start.item()
        end = members.iloc[mem].end.item()
        memb_number = members.iloc[mem].member.item()

        K = local_and_global_stiffness_matrix(memb_number)[1]
        if n == start:
            kii = K[0:3,0:3]
            kij = K[0:3,3:6]
            K_str[(n*ncjt)-3:n*ncjt,(start*ncjt)-3:start*ncjt] = K_str[(n*ncjt)-3:n*ncjt,(start*ncjt)-3:start*ncjt]+kii
            K_str[(n*ncjt)-3:n*ncjt,(end*ncjt)-3:end*ncjt] = K_str[(n*ncjt)-3:n*ncjt,(end*ncjt)-3:end*ncjt]+kij
        else:
            kji = K[3:6,0:3]
            kjj = K[3:6,3:6]
            K_str[(n*ncjt)-3:n*ncjt,(start*ncjt)-3:start*ncjt] = K_str[(n*ncjt)-3:n*ncjt,(start*ncjt)-3:start*ncjt]+kji
            K_str[(n*ncjt)-3:n*ncjt,(end*ncjt)-3:end*ncjt] = K_str[(n*ncjt)-3:n*ncjt,(end*ncjt)-3:end*ncjt]+kjj

K_str

NameError: name 'np' is not defined

In [151]:
joint_load_vector() - structure_equivalent_joint_loadvector()

array([[ 6.18056944e-05],
       [-6.00000899e+01],
       [ 7.48501771e+02]])